<u><b><font size = 6>A2: Team Project OJ </f></b></u><br><br>
<b><font size = 5> Data Optimization - DAT-5304 </f></b>
<br><br>
<b>Authors</b>          : Carolina Novello Moreira, Jack Daoud & Max Lembke <br>
<b>Date Created</b> : 03/24/2021<br>

## Set-up 

In [1]:
# import packages 
import pandas as pd #pandas 
import numpy  as np #numpy 
from pulp import * #pulp everything 
import statsmodels.formula.api as smf # predictive modeling with nice outputs
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from regressors import stats

# set options 
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 50)

# importing file 
df = pd.read_excel('OJ_Data.xlsx')

In [2]:
# helper functions 

# make binary function 
def make_binary(df):
    
    for col in df.columns:
        
        if 'feat' in col \
        or 'disp' in col:
            
            for index, value in df.iterrows():
                
                if df.loc[index, col] >= 0.5:
                    df.loc[index, col] = 1
                else:
                    df.loc[index, col] = 0
        
            df[col] = df[col].astype(int)

# rev and profit function 
def get_rev_and_profit(df):

    # create Revenue columns for each SKU
    for i in range(1,6):
        df['revenue' + str(i)] = 0
     
    
    # compute Revenue using price x sales
    for index, value in df.iterrows():
        for i in range(1,6):
            df.loc[index, 'revenue' + str(i)] \
            = df.loc[index, 'price' + str(i)] * df.loc[index, 'sales' + str(i)]
            
            df['revenue'+str(i)] = df['revenue' + str(i)].round(2)
     
    
    # create Profit columns for each SKU
    for i in range(1,6):
        df['profit' + str(i)] = 0
     
    
    # compute Profit using revenue x gross margin
    for index, value in df.iterrows():
        for i in range(1,6):
            df.loc[index, 'profit' + str(i)] = \
            df.loc[index, 'revenue' + str(i)] * df.loc[index, 'grmar' + str(i)]
            
            df['profit' + str(i)] = df['profit' + str(i)].round(2)

## Data Cleaning 

In [3]:
# data cleaning 

# calling make binary function 
make_binary(df)

# calc. rev and profits function 
get_rev_and_profit(df)

# calling df
df

,week,sales1,sales2,sales3,sales4,sales5,price1,price2,price3,price4,price5,disp1,disp2,disp3,disp4,disp5,feat1,feat2,feat3,feat4,feat5,grmar1,grmar2,grmar3,grmar4,grmar5,revenue1,revenue2,revenue3,revenue4,revenue5,profit1,profit2,profit3,profit4,profit5
0,40,6528,8448,47040,4224,2432,0.057188,0.060312,0.029531,0.046719,0.024844,0,0,0,0,0,0,0,1,0,1,0.344357,0.295700,0.107935,0.307700,0.010100,373.32,509.52,1389.15,197.34,60.42,128.56,150.67,149.94,60.72,0.61
1,43,6016,8352,3008,5696,2112,0.057188,0.060312,0.044375,0.046719,0.038906,0,0,0,0,0,0,0,1,0,0,0.342101,0.295700,0.416802,0.365920,0.428100,344.04,503.73,133.48,266.11,82.17,117.70,148.95,55.63,97.38,35.18
2,44,6272,7776,2816,36160,896,0.057188,0.060312,0.044375,0.040469,0.038906,0,0,0,0,0,0,0,0,1,0,0.319928,0.295700,0.372591,0.334818,0.428100,358.68,468.99,124.96,1463.35,34.86,114.75,138.68,46.56,489.96,14.92
3,45,6848,7968,28352,3200,1152,0.057188,0.060312,0.037344,0.046719,0.038906,0,0,0,0,0,0,0,0,0,0,0.264671,0.295700,0.257612,0.404416,0.368700,391.62,480.57,1058.77,149.50,44.82,103.65,142.10,272.75,60.46,16.53
4,46,7424,7296,3712,11904,3072,0.057188,0.060312,0.044375,0.046719,0.038906,0,0,0,0,0,0,0,0,0,0,0.260983,0.295700,0.363762,0.412581,0.374700,424.56,440.04,164.72,556.14,119.52,110.80,130.12,59.92,229.45,44.78
5,47,6848,7200,11136,21376,4672,0.057188,0.060312,0.044375,0.037344,0.032656,0,0,0,0,0,0,0,0,1,1,0.259800,0.295700,0.365395,0.207690,0.255000,391.62,434.25,494.16,798.26,152.57,101.74,128.41,180.56,165.79,38.91
6,48,7488,7968,35904,3648,1920,0.057188,0.060312,0.037344,0.046719,0.032656,0,0,0,0,0,0,0,1,0,0,0.259300,0.295700,0.221557,0.344168,0.256900,428.22,480.57,1340.79,170.43,62.70,111.04,142.10,297.06,58.66,16.11
7,49,6336,7200,2368,46848,896,0.057188,0.060312,0.037344,0.031094,0.032656,0,0,1,1,1,0,0,0,1,0,0.259000,0.338580,0.246486,0.013690,0.247800,362.34,434.25,88.43,1456.68,29.26,93.85,147.03,21.80,19.94,7.25
8,50,6208,11712,45824,3328,1152,0.057188,0.060312,0.031094,0.046719,0.032656,0,1,0,0,1,0,1,1,0,0,0.259000,0.382092,0.151579,0.351496,0.251200,355.02,706.38,1424.84,155.48,37.62,91.95,269.90,215.98,54.65,9.45
9,51,6400,10368,3008,4096,26432,0.057188,0.060312,0.044375,0.046719,0.029531,0,1,0,0,1,0,0,0,0,1,0.259000,0.373050,0.395757,0.352683,0.200000,366.00,625.32,133.48,191.36,780.57,94.79,233.28,52.83,67.49,156.11


In [4]:
# log transformations 

# price
df['log_price1'] = np.log10(df['price1'])
df['log_price2'] = np.log10(df['price2'])
df['log_price3'] = np.log10(df['price3'])
df['log_price4'] = np.log10(df['price4'])
df['log_price5'] = np.log10(df['price5'])

# sales
df['log_sales1'] = np.log10(df['sales1'])
df['log_sales2'] = np.log10(df['sales2'])
df['log_sales3'] = np.log10(df['sales3'])
df['log_sales4'] = np.log10(df['sales4'])
df['log_sales5'] = np.log10(df['sales5'])

## Regression 

### Product 1 Non-Log Max Approach 

In [5]:
# regression for product 1 

# declaring set of x-variables
x_variables = ['price1', 'price2', 'price3','price4',
               'price5','disp1','feat1']


# looping to make x-variables suitable for statsmodels
for val in x_variables:
    print(f"{val} +")

price1 +
price2 +
price3 +
price4 +
price5 +
disp1 +
feat1 +


In [6]:
# step 1: build a model
lm_best = smf.ols(formula =  """sales1 ~ price1 +
                                        price2 +
                                        price3 +
                                        price4 +
                                        price5 +
                                        disp1 +
                                        feat1""",
                                        data = df)


# step 2: fit the model based on the data
results = lm_best.fit()

# step 3: analyze the summary output
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 sales1   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     32.24
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           9.42e-24
Time:                        14:57:54   Log-Likelihood:                -1248.1
No. Observations:                 116   AIC:                             2512.
Df Residuals:                     108   BIC:                             2534.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5.243e+04   1.21e+04      4.342      0.0

### Product 1 Non-Log Jack Approach

In [7]:
# instantiate independant variables
X = df.loc[:,x_variables]

# instantiate dependant variable
y = df.loc[:, 'sales1']

# instantiate linear regression class
lm = LinearRegression()

# fit 
model = lm.fit(X,y)

# print model summary
# instantiate list of used explarotary variable names

xlabels = X.columns

# Print model output:
print(f"""
=================== MODEL SUMMARY ===================


Model Type: Ordinary Least Squares (OLS)

Model Size: {len(xlabels) + 1}

""")

stats.summary(model, X, y, xlabels)

# Code for SKLEARN p-values comes from:
# https://stackoverflow.com/questions/44495667/calculate-p-value-in-sklearn-using-python


=================== MODEL SUMMARY ===================


Model Type: Ordinary Least Squares (OLS)

Model Size: 8


Residuals:
       Min        1Q     Median         3Q        Max
-3315.3525 5940.1313 14872.6309 30523.5127 64110.9918


Coefficients:
                Estimate     Std. Error  t value   p value
_intercept  5.243066e+04    2318.430014  22.6147  0.000000
price1     -1.555457e+06  180274.621072  -8.6283  0.000000
price2      2.612364e+05  211477.197371   1.2353  0.219239
price3      5.440978e+05  174035.234900   3.1264  0.002241
price4      2.022695e+04  189554.016753   0.1067  0.915206
price5      5.491028e+04  197979.172999   0.2774  0.782006
disp1      -3.155618e+03    2197.625336  -1.4359  0.153739
feat1       2.017194e+04    2983.623822   6.7609  0.000000
---
R-squared:  0.67633,    Adjusted R-squared:  0.65535
F-statistic: 32.24 on 7 features


### Product 1 Max Approach 

In [8]:
# regression for product 1 

# declaring set of x-variables
x_variables = ['log_price1', 'log_price2', 'log_price3','log_price4',
               'log_price5','disp1','feat1']


# looping to make x-variables suitable for statsmodels
for val in x_variables:
    print(f"{val} +")

log_price1 +
log_price2 +
log_price3 +
log_price4 +
log_price5 +
disp1 +
feat1 +


In [10]:
# step 1: build a model
lm_best = smf.ols(formula =  """log_sales1 ~ log_price1 +
                                        log_price2 +
                                        log_price3 +
                                        log_price4 +
                                        log_price5 +
                                        disp1 +
                                        feat1""",
                                        data = df)


# step 2: fit the model based on the data
results = lm_best.fit()

# step 3: analyze the summary output
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             log_sales1   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     59.46
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           3.56e-34
Time:                        15:06:01   Log-Likelihood:                 57.156
No. Observations:                 116   AIC:                            -98.31
Df Residuals:                     108   BIC:                            -76.28
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4345      0.482      2.977      0.0

## Monte Carlo

### Product 1 
Note: Using the log reg. output above

In [ ]:
# set up parameters 

# price 1 
p1_mu = -2.6540
p1_sigma = 0.232

# price 2 
p2_mu = -2.6136
p2_sigma = 0.185

# price 3
p3_mu = -2.6136
p3_sigma = 0.185

# price 4 
p4_mu = -2.6136
p4_sigma = 0.185

# price 5 
p_mu = -2.6136
p5_sigma = 0.185

# disp 1 
dp1_mu = -2.6136
dp1_sigma = 0.185

# feat 1
f1_mu = -2.6136
f1_sigma = 0.185

# set up Simulation parameters
np.random.seed(42)  # fix the random seed for reproducibility 
number_sims = 1_000

all_stats = [] # Save results from each sim here

# begin Simulation Loop
for i in range(number_sims):
    c_L_current = np.random.normal(c_L_mu, c_L_sigma)
    c_P_current = np.random.uniform(c_P_min, c_P_max)
    d_current = np.random.normal(d_mu, d_sigma)
    profits_current = (p-c_L_current - c_P_current)*d_current - fixed_costs
    all_stats.append([c_L_current, c_P_current, d_current, profits_current])
                    
# save results in the results dataframe
results_df = pd.DataFrame.from_records(all_stats, columns=['c_L', 'c_P', 'd','profits'])